# UPS Stochastic MV Framework

### Import Libraries

In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.analytics.stoch_mv import stochMV
from AssetAllocation.reporting import reports as rp, plots, formatter as fmt
import numpy as np
from ipywidgets import interact, interact_manual

PLAN = 'IBT'

### Model liability of Plan

In [ ]:
liab_model = summary.get_liab_model(PLAN)

### Compute plan inputs

In [ ]:
pp_inputs = summary.get_pp_inputs(liab_model,PLAN)

### Initialize plan object and display relevant variables

In [ ]:
plan = summary.get_plan_params(pp_inputs)

#Display plan inputs
pp_dict = plan.get_pp_dict()
@interact
def display_pp_inputs(variable=pp_dict.keys()):
    if variable == 'Corr' or variable == 'Cov':
        return plots.draw_heatmap(pp_dict[variable], half=False)
    else:
        return fmt.get_plan_styler(pp_dict[variable])

### Initialize stoch MV object, generate random returns and resampled correaltions

In [ ]:
#initialize the stochastic mean variance and generate the random returns
num_of_plans = 120
s = stochMV(plan, num_of_plans)
s.generate_plans()
s.generate_resamp_corr_dict()

### View Simulated Return Data

In [ ]:
fmt.get_plan_styler(s.returns_df)

### View Resampled Correlations

In [ ]:
@interact
def display_resampled_corr(asset_liab=s.resamp_corr_dict.keys()):
    resamp_corr_fig = plots.get_resamp_corr_fig(s.resamp_corr_dict[asset_liab], asset_liab)
    resamp_corr_fig.show()

### View Simulated Returns Plots

In [ ]:
#visualize the simulated returns
plots.get_sim_return_fig(s)

### Define bounds
* Bounds data is stored in **'data/plan_inputs/bounds.xlsx'** by plan

In [ ]:
bnds = dm.get_bounds(liab_model,plan=PLAN)

@interact
def view_update_bnds(asset = plan.bnds_dict['asset_list'],reset=False,
                     lower=plan.bnds_dict['lower_bnd'],upper=plan.bnds_dict['upper_bnd'],reset_asset=False):
    if lower:
        fmt.update_lower_bnds(bnds, asset,lower,plan)
    if upper:
        fmt.update_upper_bnds(bnds, asset,upper,plan)
    if reset:
        fmt.reset_bnds(bnds,PLAN)
    if reset_asset:
        fmt.reset_asset_bnds(bnds,asset,PLAN)
    return fmt.get_plan_styler(bnds)

### Define contraints to optimize for min and max return
* Current constraints:
    * 45% <= sum(Fixed Income Assets weights) <= 55%
    * sum(All plan assets weights (excluding Futures & Hedges)) = Funded Status Diff
    * Hedges weights <= sum(50% of Equity & Private Equity weights)
    * 15+ STRIPS weight >= sum(50% of Futures and 25% of Hedges weights)

In [ ]:
cons = (
        # 45% <= sum of Fixed Income Assets <= 55%
        {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.45*s.init_plan.funded_status},
        {'type': 'ineq', 'fun': lambda x: .55*s.init_plan.funded_status - np.sum(x[1:3])},
        #sum of all plan assets (excluding Futures and Hedges) = Funded Status Difference    
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(s.init_plan)-1]) - x[3] + (1-s.init_plan.funded_status)},
        # 50% of Equity and PE >= Hedges
        {'type': 'ineq', 'fun': lambda x: (x[4]+x[6])*.5 - x[len(s.init_plan)-1]},
        # 15+ STRIPS >= sum(50% of Futures and 25% of Hedges weights)
        {'type': 'ineq', 'fun': lambda x: x[1] - (x[3]/2+x[len(s.init_plan)-1]/4)}
        )

### Compute MV Efficient Frontier for each of the simulated plans

In [ ]:
# computation of the efficient frontier for each of the simulated plan
s.generate_efficient_frontiers(bnds, cons,num_ports=100)

### Display stoch MV Efficient Frontier portfolios

In [ ]:
# output the dataframe which has the average allocation
fmt.get_port_styler(s.opt_ports_df)

### Display MV Asset Allocation

In [ ]:
#Asset Allocation Plot
aa_fig = plots.get_aa_fig(s.opt_ports_df)
aa_fig.show()

### Display MV Efficient Frontier

In [ ]:
#Plotly version of the Efficient Frontier plot
ef_fig = plots.get_ef_fig(s.opt_ports_df)
ef_fig.show()

### Export data to excel

In [ ]:
rp.get_stochmv_ef_portfolios_report(PLAN+' stochmv_ef_report', s,bnds)